In [67]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Dropout, Add
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np
from sklearn.model_selection import train_test_split

In [68]:
tf.__version__

'2.10.0'

In [69]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) == 0:
    print("No GPU devices found. Using CPU.")
else:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Data Pre-Processing

Open **play_style_train.csv** file and split the games into a list.
Every row of csv: `PSL0000000001,1,B[pd],W[dp],B[qp],W[dc],B[nq],W[nc],B[qf],W[kd],B[ce],W[dg],B[dd],W[cc],B[fd],W[ed],B[ee],W[ec],B[ge],W[gc],B[di]`. 

Columns are:

    1. PSL0000000001: Game ID
    2. 1: Game Style
    3-... : Moves, the last move represents the play style (B[di] in this case)
    
We cropped only the moves to game list as:

In [70]:
import random
df = open('./Training Dataset/play_style_train.csv').read().splitlines()
random.shuffle(df)
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]
print(games[0], game_styles[0])

B[pd],W[dp],B[dd],W[qp],B[op],W[pn],B[qq],W[rq],B[rr],W[pq],B[qr],W[pp],B[rp],W[ro],B[sq],W[pr],B[fq],W[jp],B[cq],W[dq],B[cp],W[do],B[dr] 3


Create a dictionary to convert the coordinates from characters to numbers

In [71]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 2 dimensional feature map to represent the data as below:
 1. Occupied areas: mark them as 1 and the empty places as 0
 2. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
The target is to predict the game style (1, 2 or 3) from the state of the game table. Later this will be one-hot encoded.

In [72]:
# def prepare_input(moves):
#     x = np.zeros((19,19,2))
#     for move in moves:
#         color = move[0]
#         column = coordinates[move[2]]
#         row = coordinates[move[3]]
#         x[row,column,0] = 1
#     if moves:
#         last_move_column = coordinates[moves[-1][2]]
#         last_move_row = coordinates[moves[-1][3]]
#         x[row,column,1] = 1
#     return x

In [73]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row,last_move_column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

In [74]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 26615


Since play style training has smaller dataset comparing to kyu or dan training, we can put the complete dataset to memory. Still, it is better to create a data generator.

In [75]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(prepare_input(moves_list))
x = np.array(x)
y = np.array(game_styles)-1

In [76]:
x.shape

(26615, 19, 19, 4)

In [77]:
y.shape

(26615,)

In [78]:
np.bincount(y)

array([8184, 9403, 9028], dtype=int64)

Target is one-hot encoded and loss is changed to `categorical_crossentropy`

In [79]:
y_hot = tf.one_hot(y, depth=3)

Dataset splitting: 90% Training, 10% validation

In [80]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot.numpy(), test_size=0.1)

# Training

### Simple DCNN Model:

In [81]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import gc
import math
import time

In [82]:
%load_ext tensorboard
%tensorboard --logdir=path/to/logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 10952), started 5 days, 6:55:35 ago. (Use '!kill 10952' to kill it.)

In [83]:
def create_model():
    inputs = Input(shape=(19, 19, 2))
    outputs = Conv2D(kernel_size=7, filters=256, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=256, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=256, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=256, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Flatten()(outputs)
    
    outputs = Dense(32, activation='relu')(outputs)
    outputs = Dropout(0.6)(outputs)
    outputs = Dense(3, activation='softmax', )(outputs)

    model = Model(inputs, outputs)
    opt = Adam(learning_rate=0.00005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [84]:
# def create_model():
#     inputs = Input(shape=(19, 19, 2))
#     outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Flatten()(outputs)
#     outputs = Dense(32, activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Dense(32, activation='relu')(outputs)
#     outputs = BatchNormalization()(outputs)
#     outputs = Dense(3, activation='softmax', )(outputs)
#     model = Model(inputs, outputs)
#     opt = Adam(learning_rate=0.00005)
#     model.compile(optimizer=opt,
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])
#     return model

In [85]:
def create_model2():
    inputs = Input(shape=(19, 19, 4))

    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    
    outputs = Flatten()(outputs)
    outputs = Dense(256, activation='relu')(outputs)
    outputs = Dropout(0.6)(outputs)
    outputs = Dense(3, activation='softmax')(outputs)  # Adjust the number of units based on your output space
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.001)  # Adjust the learning rate
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',  # Change to sparse_categorical_crossentropy
                  metrics=['accuracy'])

    return model

In [86]:
model = create_model2()
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d_52 (Conv2D)          (None, 19, 19, 64)        12608     
                                                                 
 batch_normalization_55 (Bat  (None, 19, 19, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_53 (Conv2D)          (None, 19, 19, 64)        200768    
                                                                 
 batch_normalization_56 (Bat  (None, 19, 19, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_54 (Conv2D)          (None, 19, 19, 64)        1024

In [87]:

tensorboard_callback = TensorBoard(log_dir=f"playstlye_logs/{time.time()}", histogram_freq=1)
# Define a custom callback to save model based on validation accuracy
class SaveModelCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        filepath = f"./pl/{logs['val_loss']:.3f}_{logs['val_accuracy']:.3f}.h5"
        self.model.save(filepath)

# Create an instance of the SaveModelCallback
save_model_callback = SaveModelCallback()

history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 16,
    epochs = 30,
    validation_data=(x_val, y_val),
    callbacks=[tensorboard_callback, save_model_callback],
)

Epoch 1/30
1647/1647 [==============================] - 15s 9ms/step - loss: 1.1164 - accuracy: 0.3845 - val_loss: 1.1277 - val_accuracy: 0.3071
Epoch 2/30
1647/1647 [==============================] - 15s 9ms/step - loss: 1.0630 - accuracy: 0.4249 - val_loss: 1.0433 - val_accuracy: 0.4944
Epoch 3/30
1647/1647 [==============================] - 14s 9ms/step - loss: 0.9874 - accuracy: 0.5133 - val_loss: 0.9514 - val_accuracy: 0.5206
Epoch 4/30
1647/1647 [==============================] - 15s 9ms/step - loss: 0.9711 - accuracy: 0.5277 - val_loss: 0.9580 - val_accuracy: 0.5206
Epoch 5/30
1647/1647 [==============================] - 15s 9ms/step - loss: 0.9613 - accuracy: 0.5356 - val_loss: 0.9388 - val_accuracy: 0.5318
Epoch 6/30
1647/1647 [==============================] - 15s 9ms/step - loss: 0.9555 - accuracy: 0.5393 - val_loss: 0.9258 - val_accuracy: 0.5693
Epoch 7/30
1647/1647 [==============================] - 15s 9ms/step - loss: 0.9482 - accuracy: 0.5441 - val_loss: 0.9413 - val_ac

KeyboardInterrupt: 

In [ ]:
# model.save(f"./model_playstyle{history.history['val_accuracy'][-1]:.5f}.h5")

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!